# Analyse exploratoire


**1 - Importation des modules**

In [2]:
import pandas as pd
import numpy as np

from utils.data import get_gender, get_states
from utils.coherence import correct_typo, find_best_similar, gestalt_pattern_matching
from utils.coherence import calculate_age, convert_to_date
from utils.eda import apply_gender

**2 - Lecture des données**

In [3]:
df = pd.read_csv('df_patient_pcr.csv')
df = df.replace({np.nan: None})

In [4]:
df.head()

,patient_id,pcr,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2,born_age,localisation,full_address,full_name
0,653091,N,daniel,campbell,58,sid barnes crescent,north ward,4514,nsw,1.97304e+07,None,08 38772117,None,19730426,4514 nsw north ward,58 sid barnes crescent,campbell daniel
1,347667,N,sebastian,mchenry,12,mundawari circuit,swan view,4551,wa,1.99003e+07,28,08 61083524,ocean star villas,19900316 28,4551 wa swan view,12 mundawari circuit,mchenry sebastian
2,708481,P,sarah,rellos,20,torres street,bribie island,3199,sa,1.94303e+07,None,02 73197286,None,19430317,3199 sa bribie island,20 torres street,rellos sarah
3,148825,N,chloe,brammy,238,fitchett street,carnegie,3280,qld,1.95405e+07,19,02 48826642,None,19540528 19,3280 qld carnegie,238 fitchett street,brammy chloe
4,150081,N,charlie,None,6,clark close,south melbourne,2602,qld,1.97503e+07,35,03 24096098,talawa,19750331 35,2602 qld south melbourne,6 clark close,charlie


## Pré-traitement des données

Avant de commencer l'analyse exploratoires, nous allons nettoyer la base de données et créer de nouvelles variables pouvant être pertinentes lors de l'analyse.  Enfin, nous sélectionnerons les variables utiles pour l'analyse pour créer notre jeu de données final.

### Création d'une variable `Gender`

On commence par corriger les prénoms mal orthographiés avec la fonction utilisée précédemment pour les adresses et la banlieue (`correct_typo`).

Après plusieurs itérations, une similtude à 85% entre deux chaînes de caractère semble convenir en prenant comme base de référence les prénoms apparaissant plus de 3 fois (pour ne pas corriger un prénom par un autre).

In [4]:
given_name_corrected = correct_typo(df.given_name, threshold=3,confidence=85)
df.given_name = df.given_name.replace(given_name_corrected)

On utilise ensuite une [API](https://genderize.io) qui fournit le sexe s'apparentant au mieux à un prénom, avec une probabilité indiquant la confiance de cette prédiction.

*NOTE : Toutefois, l'API limite ses requêtes et une exception est levée lors du seuil par jour atteind. Dans ce cas, la fonction renvoie donc la dernière données sauvegardée.*

In [5]:
unique_name = df.given_name.unique()
all_gender = get_gender(unique_name)

Request limit


On a donc notre nouvelle variable `gender`.

In [6]:
df["gender"] = apply_gender(all_gender, apply=df.given_name)

### Création de la variable `class_age`

Une seconde étape consiste à mettre en forme les données avant de créer une variable `class_age` calculée selon deux variable : `age` et `date_of_birth`.

In [5]:
df["age_estimated"] = calculate_age(df.date_of_birth, year=2020)
df["age"] = df.age.astype(float)

Pour appliquer la transformation en tranche d'âge (car il y a beaucoup d'erreurs de cohérence entre la date de naissances et l'age renseigné), on utilise le découpage effectué par les documents d'analyse de **Santé Publique France**.

In [58]:
df["age_group"] = df.apply(
    lambda x : find_truth_age(x["age"], x["age_estimated"]), 
    axis=1)

df["age_group"] = df.age_group.replace({None: np.nan})

Pour chaque ligne, on compare simultanément **l'age estimé** et **l'age renseigné**.
- si les deux valeurs concordent (se trouvent dans la même classe d'âge) alors on applique la tranche d'age pour cette valeur.
- si une des deux valeurs est manquante, on impute la classe d'age à partir de l'autre valeur présente (en vérifiant la concordance).
- si les deux valeurs ne concordent pas (se trouve dans des classes d'âges différentes) alors on peut :

     - prendre la moyenne des valeurs et imputer pour cette nouvelles classe (mais cela peut fausser grandement les analyses).
     - vérifier la cohérence de chacune des valeurs et appliquer la plus probable (mais lorsque les deux valeurs sont probables, il est très difficile d'appliquer une règle).


In [67]:
round(df.age_group.isna().sum() / df.shape[0],3)

0.603

Seulement 40% (2886) des valeurs concordent lorsque nous découpons sous forme de classes entre la variable `age` et la variable `age_estimated` calculée à partir de la variable `date_of_birth`.

In [73]:
df[['age','age_estimated','date_of_birth','age_group']][
    df.age_group.isna()].sample(10)

,age,age_estimated,date_of_birth,age_group
6387,27.0,88,19321217.0,NaN
7178,22.0,98,19220618.0,NaN
2600,28.0,64,19560525.0,NaN
5379,38.0,112,19081209.0,NaN
6118,22.0,65,19551229.0,NaN
4102,13.0,37,19830123.0,NaN
209,37.0,51,19690524.0,NaN
6412,24.0,45,19750606.0,NaN
230,37.0,75,19450114.0,NaN
2294,28.0,46,19741021.0,NaN


On peut voir que sur un échantillon, les différences entre `age` et `age_estimated` ne font ressortir aucun *pattern*. Il vaut donc mieux laisser ce champ inconnu.

*NB : petite idée, trouver une base comprennant la moyenne d'âge pour un prénom et de réperer quel est l'age le plus concordant avec cette moyenne et imputer selon cette valeur.*

### Nettoyage des variables : `postcode` et `state`

Nous appliquons ici les étapes que nous avons repérées lors du dernier notebook (correction des erreurs de typographie pour l'état et la vérification des codes postaux).

In [74]:
dict_ref = get_states()
df["state"] = find_best_similar(df.state, dict_ref, gestalt_pattern_matching)